In [2]:
import pandas as pd
import sqlalchemy as sq
import numpy as np

Create a connection to the database. This will eventually create a .db file in the directory of the notebook once we write something to the connection.

In [33]:
engine = sq.create_engine('sqlite:///OKCdatabase.db')

Load the dataframes

In [3]:
#main dataframe of user answers
df_main = pd.read_csv('data/user_data_public.csv',index_col=None, dtype=str)
df_main.head()

,q2,q11,q12,q13,q14,q16,q17,q18,q20,q22,...,q86364,q86397,q86462,q86615,q86699,q363047,CA,gender_orientation,gender,race
0,NaN,Horrified,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.571282953049484,Hetero_female,Woman,White
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hetero_male,Man,NaN
2,NaN,NaN,NaN,No,No,NaN,No,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.01226429848596,Hetero_female,Woman,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hetero_female,Woman,White
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.418363901130365,Bisexual_female,Woman,NaN


In [35]:
# add userID to main
df_main['userID'] = np.arange(1, df_main.shape[0]+1)

In [10]:
df_main.q34113.describe()

count                 68371
unique                    4
top       Never - Get a job
freq                  37271
Name: q34113, dtype: object

In [36]:
#dataframe of general questions
df_qs = pd.read_csv('question_data.csv',sep=';',index_col=None)
df_qs = df_qs.sort_values('N',ascending=False);

In [37]:
#dataframe of cognitive test questions
df_ts = pd.read_csv('test_items.csv',index_col=None)

Examining the questions set, we see that there are 5 types of questions:

1. Personality questions, denoted by columns named qXXX (where X is numeric)
2. Test questions, also denoted by columns named qXXX
3. User's match preferences, denoted by columns named lf_ZZZZ (where Z is alphabetic)
4. User attributes, denoted by columns named d_ZZZZ
5. OKC personality scale scores, denoted by columns named p_ZZZZ

Since different question types have different kinds of answers, we will break the main table into 5 smaller tables

In [38]:
#list of all of the question headers (sorted by number of responses):
headers_all = list(df_qs.iloc[:,0])
#list of headers that starts with q
headers_q = [name for name in headers_all if (name[0] == 'q')]
#list of headers that starts with If
headers_lf = [name for name in headers_all if (name[0] == 'l')]
#list of headers that starts with d
headers_d = [name for name in headers_all if (name[0] == 'd')]
#list of headers that starts with p
headers_p = [name for name in headers_all if (name[0] == 'p')]
#list of cognitive test question headers
headers_ts = list(df_ts.iloc[:,0])
#list of non-cognitive test question headers
headers_qns = [name for name in headers_q if (name not in headers_ts)]

In [39]:
#OKC Personality Scores Table:
tab_per = df_main.loc[:,headers_p]

In [40]:
#User Attributes Table:
tab_att = df_main.loc[:,headers_d]

In [41]:
#User Match Preferences Table:
tab_prf = df_main.loc[:,headers_lf]

In [42]:
#Test Questions Table:
tab_tst = df_main.loc[:,headers_ts]

The table to store other questions need to be broken up into smaller chunks. We'll just store the top 1000 in 5 tables, and use the first one for our MVP. We can add more features to [possibly] improve the model scores afterwards. 

In [43]:
#Other Questions Table (top 200):
tab_qn1 = df_main.loc[:,headers_qns[0:200]]

In [44]:
#Other Questions Table (200-400):
tab_qn2 = df_main.loc[:,headers_qns[200:400]]

In [45]:
#Other Questions Table (400-600):
tab_qn3 = df_main.loc[:,headers_qns[400:600]]

In [46]:
#Other Questions Table (600-800):
tab_qn4 = df_main.loc[:,headers_qns[600:800]]

In [47]:
#Other Questions Table (800-1000):
tab_qn5 = df_main.loc[:,headers_qns[800:1000]]

Table for the 2 questions keys tables

In [48]:
#Test Questions Key Table:
tab_keyq = df_qs
#Other Questions Key Table:
tab_keyt = df_ts

In [49]:
#Cognitive Score Table
tab_CA = df_main.CA

In [50]:
# add userID to all derrived tabels before creating DB
tab_CA['userID'] = np.arange(1, tab_CA.shape[0]+1)
tab_per['userID'] = np.arange(1, tab_per.shape[0]+1)
tab_prf['userID'] = np.arange(1, tab_prf.shape[0]+1)
tab_att['userID'] = np.arange(1, tab_att.shape[0]+1)
tab_tst['userID'] = np.arange(1, tab_tst.shape[0]+1)
tab_qn1['userID'] = np.arange(1, tab_qn1.shape[0]+1)
tab_qn2['userID'] = np.arange(1, tab_qn2.shape[0]+1)
tab_qn3['userID'] = np.arange(1, tab_qn3.shape[0]+1)
tab_qn4['userID'] = np.arange(1, tab_qn4.shape[0]+1)
tab_qn5['userID'] = np.arange(1, tab_qn5.shape[0]+1)

/Users/kylemix/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [51]:
tab_tst.shape

(68371, 29)

Now we write these tables into the SQL database connection set up by the 'create_engine' statement

In [52]:
tab_per.to_sql('personality_scores',engine,index=False)

In [53]:
tab_CA.to_sql('cognitive_scores',engine,index=False)

In [54]:
tab_att.to_sql('user_info',engine,index=False)

In [55]:
tab_prf.to_sql('match_preferences',engine,index=False)

In [56]:
tab_tst.to_sql('test_answers',engine,index=False)

In [57]:
tab_qn1.to_sql('question_responses1',engine,index=False)

In [58]:
tab_qn2.to_sql('question_responses2',engine,index=False)

In [59]:
tab_qn3.to_sql('question_responses3',engine,index=False)

In [60]:
tab_qn4.to_sql('question_responses4',engine,index=False)

In [61]:
tab_qn5.to_sql('question_responses5',engine,index=False)